**#30DayMapChallenge 2024: Hypothetical U.S. National Parks Train Map**

This notebook is used to draw a hypothetical map of train lines between National Park using 1) nearest park if no existing line and 2) minimum spanning tree

In [2]:
import geopandas as gpd
import folium
import numpy as np
import networkx as nx
from shapely.geometry import LineString

In [3]:
#coordinates downloaded from https://en.wikipedia.org/wiki/List_of_national_parks_of_the_United_States
nationalparks = gpd.read_file("National Park Coordinates.kml", driver="KML")

In [4]:
nationalparks.drop(columns = "Description")

,Name,geometry
0,Acadia,POINT Z (-68.21 44.35 0)
1,National Park of American Samoa,POINT Z (-170.68 -14.25 0)
2,Arches,POINT Z (-109.57 38.68 0)
3,Badlands,POINT Z (-102.5 43.75 0)
4,Big Bend,POINT Z (-103.25 29.25 0)
...,...,...
58,Wind Cave,POINT Z (-103.48 43.57 0)
59,Wrangell – St. Elias,POINT Z (-142 61 0)
60,Yellowstone,POINT Z (-110.5 44.6 0)
61,Yosemite,POINT Z (-119.5 37.83 0)


In [5]:
#calculate the center of all points
mean_lat = nationalparks.geometry.y.mean()
mean_lon = nationalparks.geometry.x.mean()

#initialise the map centered at the mean coordinates
m = folium.Map(location=[mean_lat, mean_lon], zoom_start=3)

#add a marker for each point in the GeoDataFrame
for _, row in nationalparks.iterrows(): #_, used to ignore index of the row/iterrows to iterate through rows
    point = row.geometry
    name = row['Name']  #access the 'Name' column for the popup
    folium.Marker(
        location=[point.y, point.x],  #latitude, longitude from the geometry
        popup=name,                   #display 'Name' column as the popup
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(m)

#display the map
m

**Method 1: Iterate through the list and draw a line between a park and another park if it is the shortest and a line has not already been drawn**

In [7]:
#to store lines that have been drawn
lines_drawn = set()

In [8]:
#function to draw lines
def draw_line_between_parks(park_a, park_b):
    line = LineString([park_a.geometry.centroid, park_b.geometry.centroid])
    lines_drawn.add((park_a['Name'], park_b['Name']))  #add to drawn lines
    folium.PolyLine(locations=[(park_a.geometry.centroid.y, park_a.geometry.centroid.x),
                                (park_b.geometry.centroid.y, park_b.geometry.centroid.x)],
                    color='blue', weight=2.5, opacity=1).add_to(m)

In [9]:
#iterate over each national park
for i, park_a in nationalparks.iterrows():
    nearest_distance = float('inf')
    nearest_park = None

    #find the nearest park
    for j, park_b in nationalparks.iterrows():
        if park_a['Name'] == park_b['Name']:  #skip itself
            continue
        
        #check if a line has already been drawn between park_a and park_b
        if (park_a['Name'], park_b['Name']) in lines_drawn or (park_b['Name'], park_a['Name']) in lines_drawn:
            continue  #line already drawn, skip to the next
        
        distance = park_a.geometry.distance(park_b.geometry)

        #update nearest park if found a closer one
        if distance < nearest_distance:
            nearest_distance = distance
            nearest_park = park_b

    #draw the line to the nearest park if found
    if nearest_park is not None:
        draw_line_between_parks(park_a, nearest_park)

#add markers for each park
for _, row in nationalparks.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=row['Name'],
        icon=folium.Icon(color='green', icon='info-sign')
    ).add_to(m)

#display the map
m

**Method 2: Minimum Spanning Tree**

In [20]:
#create a distance matrix
num_parks = len(nationalparks)
dist_matrix = np.zeros((num_parks, num_parks))

#calculate distance between park_a and park_b
for i, park_a in nationalparks.iterrows():
    for j, park_b in nationalparks.iterrows():
        if i != j:  #skip self-distance
            dist_matrix[i, j] = park_a.geometry.distance(park_b.geometry)

#create a graph from the distance matrix
#convert distance matrix into a graph where each park is a node and the distance between them is the weight of edges
G = nx.from_numpy_array(dist_matrix)

#generate the minimum spanning tree to connect all parks/nodes with the smallest possible distance while ensuring no loops
mst = nx.minimum_spanning_tree(G)

#draw the lines for the minimum spanning tree
for i, j in mst.edges():
    park_a = nationalparks.iloc[i]
    park_b = nationalparks.iloc[j]
    
    #draw the line between park_a and park_b
    line = LineString([park_a.geometry.centroid, park_b.geometry.centroid])
    folium.PolyLine(locations=[(park_a.geometry.centroid.y, park_a.geometry.centroid.x),
                                (park_b.geometry.centroid.y, park_b.geometry.centroid.x)],
                    color='blue', weight=2.5, opacity=1).add_to(m)

#add markers for each park
for _, row in nationalparks.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=row['Name'],
        icon=folium.Icon(color='green', icon='info-sign')
    ).add_to(m)

#display the map
m